In [ ]:
# !pip install opencv-python-headless
# encountered an issue regarding its version while downloading on PC.
# Also, update all required frameworks and libraries to the latest versions to ensure compatibility.

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
#from google.colab.patches import cv2_imshow
from IPython.display import display, Javascript
#from google.colab.output import eval_js
import time
import base64

## Link to generate your custom checkerboard https://markhedleyjones.com/projects/calibration-checkerboard-collection ##

In [3]:
# Checkerboard properties
CHECKERBOARD = (8, 6)  # 8x6 vertices (the points between squares)
SQUARE_SIZE = 25  # Size of squares in mm

In [4]:
# Termination criteria for corner refinement
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

In [5]:
# Prepare object points (0,0,0), (1,0,0), (2,0,0), ... in world coordinates
objp = np.zeros((CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
objp *= SQUARE_SIZE  # Scale to square size

In [6]:
# Arrays to store object points and image points
objpoints = []  # 3d points in real world space
imgpoints = []  # 2d points in image plane

In [7]:
# Create directory for captured images
if not os.path.exists("calib_images"):
    os.makedirs("calib_images")

In [8]:
# JavaScript to capture an image
# cv.read (for reading the camera)
# designed for usage via browser
# define the port and the device wihle using offline
def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture Image';
      div.appendChild(capture);
      document.body.appendChild(div);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      div.appendChild(video);
      video.srcObject = stream;
      await new Promise((resolve) => video.onloadedmetadata = resolve);
      video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getTracks().forEach(track => track.stop());
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = base64.b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename

In [ ]:
# Capture 10 images
# Take photo does not work.
for i in range(10):
    filename = f'calib_images/image_{i+1}.jpg'
    print(f'Capturing {filename}')
    take_photo(filename)
    time.sleep(2)  # Pause to allow user to position the checkerboard

Capturing calib_images/image_1.jpg


<IPython.core.display.Javascript object>

NameError: name 'eval_js' is not defined

In [10]:
# Load captured images
images = [f'calib_images/image_{i+1}.jpg' for i in range(10)]

## Inititate Image Processing Algorithms ##

In [ ]:
# Gray conversion does not work.
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, None)

    if ret:
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)
        cv2_imshow(img)

error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [ ]:
# Perform camera calibration
ret, camera_matrix, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

In [ ]:
print("Camera Matrix:")
print(camera_matrix)
print("\nDistortion Coefficients:")
print(dist_coeffs)

In [ ]:
# Undistort a sample image
img = cv2.imread(images[2]) # Choose the picture (0,9).
h, w = img.shape[:2]
new_camera_matrix, roi = cv2.getOptimalNewCameraMatrix(camera_matrix, dist_coeffs, (w, h), 1, (w, h))
undistorted_img = cv2.undistort(img, camera_matrix, dist_coeffs, None, new_camera_matrix)

# Show original vs undistorted image
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
axs[0].set_title("Original Image")
axs[0].axis("off")

axs[1].imshow(cv2.cvtColor(undistorted_img, cv2.COLOR_BGR2RGB))
axs[1].set_title("Undistorted Image")
axs[1].axis("off")

plt.show()

## Object Size Estimation ##

In [ ]:
# ---- Object Size Estimation ----
def measure_object_size(image_path, camera_matrix, dist_coeffs, square_size=SQUARE_SIZE):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    undistorted_img = cv2.undistort(img, camera_matrix, dist_coeffs, None, camera_matrix)

    # Edge detection
    edges = cv2.Canny(undistorted_img, 50, 150)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        print("No object detected.")
        return

    # Find the largest contour (assumed to be the marker)
    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)

    # Calculate real-world size based on checkerboard square size
    pixel_per_mm = w / (CHECKERBOARD[0] * square_size)
    real_width = w / pixel_per_mm
    real_height = h / pixel_per_mm

    print(f"Estimated Object Size (Width, Height): {real_width:.2f} mm x {real_height:.2f} mm")

    # Draw bounding box and display
    cv2.rectangle(undistorted_img, (x, y), (x+w, y+h), (0, 255, 0), 2)
    cv2_imshow(undistorted_img)

# Apply object size measurement to an image
measure_object_size(images[0], camera_matrix, dist_coeffs)